In [ ]:
import requests
import csv
import psycopg2

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# URL of the CSV file
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv"

# Fetch the CSV data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Decode the content and split into lines
    content = response.content.decode('utf-8').splitlines()
    
    # Read the CSV data
    reader = csv.reader(content)
    next(reader)  # Skip the header row

    # Extract symbols and write to a .txt file
    with open("sp500_symbols.txt", "w") as file:
        for row in reader:
            symbol = row[0]  # The first column contains the symbol
            file.write(symbol + "\n")
    
    print("S&P 500 symbols have been saved to sp500_symbols.txt.")
else:
    print("Failed to fetch data:", response.status_code)

S&P 500 symbols have been saved to sp500_symbols.txt.


In [2]:
import psycopg2

print(psycopg2.__version__)


2.9.11 (dt dec pq3 ext lo64)


In [ ]:

import psycopg2

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS sp500_symbols (
    symbol VARCHAR(10) primary key NOT NULL UNIQUE,
);
"""
cursor.execute(create_table_query)
conn.commit()

# Open the file and read symbols
with open("sp500_symbols.txt", "r") as file:
    symbols = file.readlines()

# Insert symbols into the database
for symbol in symbols:
    symbol = symbol.strip()  # Remove any whitespace/newline characters
    try:
        cursor.execute("INSERT INTO sp500_symbols (symbol) VALUES (%s) ON CONFLICT (symbol) DO NOTHING;", (symbol,))
    except Exception as e:
        print(f"Error inserting {symbol}: {e}")

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("S&P 500 symbols have been stored in the PostgreSQL database.")

S&P 500 symbols have been stored in the PostgreSQL database.


In [2]:
import psycopg2
import csv
import re
from datetime import datetime
import requests

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
create table IF NOT EXISTS sp500_info (
    symbol VARCHAR(10) primary key NOT NULL UNIQUE,
    security VARCHAR(255) NOT NULL,
    gics_sector VARCHAR(255) NOT NULL,
    gics_sub_industry VARCHAR(255) NOT NULL,
    headquarters_location VARCHAR(255) NOT NULL,
    date_added DATE NOT NULL,
    cik VARCHAR(20) NOT NULL,
    founded TEXT NOT NULL
);
"""
cursor.execute(create_table_query)
conn.commit()

# Load CSV data from URL
url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
# Fetch the CSV data
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    # Open a CSV file and read data
    # Decode the content and split into lines
    lines = response.content.decode('utf-8').splitlines()
    # Read the CSV data
    reader = csv.reader(lines)
    header = next(reader)  # Get the header row

    # Extract symbols and write to a .txt file
    with open("sp500_information.txt", "w", newline='', encoding='utf-8') as txt_file:
        # Optional: write the header as a comment line
        txt_file.write("# " + ",".join(header) + "\n")

        for row in reader:
            # Write the full row as a CSV line (comma-separated)
            txt_file.write(",".join(row) + "\n")

    print("All rows & columns have been saved to sp500_information.txt.")
else:
    print("Failed to fetch data:", response.status_code)

# Re-open the same data for DB insertion (re-use the same reader)
    # Reset the reader because we already consumed it above
lines = response.content.decode('utf-8').splitlines()
reader = csv.reader(lines)
next(reader)   # skip header again

# Insert data into the database
for row in reader:
    # Convert date from string to date object
    date_added = datetime.strptime(row[5], '%Y-%m-%d').date()
    
    # Clean the founded value
    founded_cleaned = re.sub(r'\s*\(.*\)', '', row[7])  # Remove anything in parentheses

    cursor.execute("""
        INSERT INTO sp500_info (symbol, security, gics_sector, gics_sub_industry, headquarters_location, date_added, cik, founded)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (symbol) DO NOTHING;
    """, (row[0], row[1], row[2], row[3], row[4], date_added, row[6], founded_cleaned))

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("S&P 500 companies have been stored in the PostgreSQL database.")

All rows & columns have been saved to sp500_information.txt.
S&P 500 companies have been stored in the PostgreSQL database.
